In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain, combinations

# Problem 1

In [2]:
################################################################################
# Part (a)

import pickle
wine = pickle.load(open('wine.pkl', 'rb'))

In [3]:
def learn(train_x, train_y, num_classes=3):
    train_x_f1 = train_x[:, 0]
    train_x_f2 = train_x[:, 1]
    return [(np.mean(train_y == k), np.mean(train_x_f1[train_y == k]), np.mean(train_x_f2[train_y == k]), np.cov(train_x_f1[train_y == k], train_x_f2[train_y == k])) for k in range(num_classes)]

In [4]:
def predict(params, test_x):
    all_log_posteriors = []
    for prior, mu1, mu2, cor in params:
        mat = np.array([[test_x[0]-mu1],[test_x[1]-mu2]])
        log_posterior = np.log(prior) - np.log(np.linalg.det(cor))/2 - (mat.T @ np.linalg.inv(cor) @ mat)/2
        all_log_posteriors.append(log_posterior)
    
    log_posteriors = np.array(all_log_posteriors)
    return np.argmax(log_posteriors, axis=0)

In [5]:
def select_params(train_data, train_label):
    optimal_feat_idx = None
    min_err_count = 10000000
    
    feat_idx = combinations(range(train_data.shape[1]), 2)
    for i in feat_idx:
        # make a deep copy
        X = np.array(train_data)

        # track number of correct predictions given the two features
        err_count = 0

        # select two features
        idx1 = slice(i[0], i[0]+1)
        idx2 = slice(i[1], i[1]+1)
        extracted = np.hstack((X[:, idx1], X[:, idx2]))

        # leave-one-out cross validation
        for j in range(extracted.shape[0]):
            val_test = extracted[j]
            val_test_label = train_label[j]
            mask = np.ones(extracted.shape[0], bool)
            mask[0] = 0
            val_train = extracted[mask]
            val_train_label = train_label[mask]
            
            model = learn(val_train, val_train_label)
            result = predict(model, val_test)[0]
            if(result != val_test_label):
                err_count += 1

        # check optimality
        if (min_err_count > err_count):
            min_err_count = err_count
            optimal_feat_idx = i
    return optimal_feat_idx

In [6]:
train_data = np.array(wine['data'].astype(float))
train_label = np.array(wine['labels'])

select_params(train_data, train_label)

(0, 6)

In [7]:
################################################################################
# Part (b)

train_data = np.array(wine['data'].astype(float))
train_label = np.array(wine['labels'])
test_data = np.array(wine['testdata'].astype(float))
test_label = np.array(wine['testlabels'])

extracted_train_data = np.column_stack((train_data[:, 0], train_data[:, 6]))
extracted_test_data = np.column_stack((test_data[:, 0], test_data[:, 6]))
model = learn(extracted_train_data, train_label)

print(model)

train_err = 0
test_err = 0

# find training error rates
for i in range(extracted_train_data.shape[0]):
    pred = predict(model, extracted_train_data[i])[0]
    if (pred != train_label[i]):
        train_err += 1

train_error_rate = train_err/train_data.shape[0]
print(train_error_rate)

# find test error rates
for i in range(extracted_test_data.shape[0]):
    pred = predict(model, extracted_test_data[i])[0]
    if (pred != test_label[i]):
        test_err += 1

test_error_rate = test_err/test_data.shape[0]
print(test_error_rate)

[(0.32954545454545453, 13.771379310344829, 3.0075862068965518, array([[0.23628374, 0.06198559],
       [0.06198559, 0.12301182]])), (0.3977272727272727, 12.220857142857144, 1.9999999999999996, array([[0.21636101, 0.05461471],
       [0.05461471, 0.44718235]])), (0.2727272727272727, 13.126666666666665, 0.7758333333333333, array([[0.24196232, 0.05349855],
       [0.05349855, 0.09212101]]))]
0.045454545454545456
0.07777777777777778


# Problem 2

In [8]:
# Pr(Y=1|X=x) = p0 if x <= t1
# Pr(Y=1|X=x) = p1 if t1 < x < t2
# Pr(Y=1|X=x) = p2 if x >= t2

t1 = 0.2
t2 = 0.8
p0 = 0.25
p1 = 0.6
p2 = 0.3

In [9]:
################################################################################
# Part (b)

# Function to implement
def stump_err(t, a, b):
    if (t<0 or t>1):
        return 0.47
    # elif(a==1 and b==1):
    #     return 0.53
    if(a==1 and b==0):
        if(t<=t1):
            return 0.75*t + 0.25*(0.2-t) + 0.42
        elif(t<t2):
            return 0.4*(t-0.2) + 0.6*(0.8-t) + 0.21
        else:
            return 0.7*(t-0.8) + 0.3*(1-t) + 0.39
    else:
        if(t<=t1):
            return 0.25*t + 0.75*(0.2-t) + 0.38
        elif(t<t2):
            return 0.6*(t-0.2) + 0.4*(0.8-t) + 0.19
        else:
            return 0.3*(t-0.8) + 0.7*(1-t) + 0.41
        

print(f'T1: {stump_err(0.4,0,1)}')
print(f'T2: {stump_err(0.5,0,1)}')
print(f'T3: {stump_err(0.5,1,0)}')

T1: 0.47000000000000003
T2: 0.49000000000000005
T3: 0.51


In [10]:
################################################################################
# Part (c)
def min_err(t):
    return min(stump_err(t,0,1), stump_err(t,1,0), stump_err(t,0,0), stump_err(t,1,1))

# Plotting code
t_vals = np.linspace(-0.2, 1.2, num=14001)
best_stump_err_rates = np.array([min_err(t) for t in t_vals])

plt.figure()
plt.plot(t_vals, best_stump_err_rates)
plt.xlabel('$t$')
plt.ylabel('best stump error rate with predicate $x \leq t$')
plt.savefig('error_rates.pdf', bbox_inches='tight')
plt.close()

# find smallest value
print(min_err(0.2))

0.43


In [11]:
################################################################################
# Part (d)

def calc_err_rate(t, a, b, x, y):
    err = 0
    if(a==1 and b==0):
        for i in range(len(y)):
            if (x[i] <= t and y[i] == 0):
                err += 1
            if (x[i] > t and y[i] == 1):
                err += 1
    else:
        for i in range(len(y)):
            if (x[i] <= t and y[i] == 1):
                err += 1
            if (x[i] > t and y[i] == 0):
                err += 1
    return err/len(y)

# Function to implement
def find_best_stump(x,y):
    t = 0
    a = 0
    b = 0
    train_err_rate = 1
    for t_val in x:
        err01 = calc_err_rate(t_val, 0, 1, x, y)
        err10 = calc_err_rate(t_val, 1, 0, x, y)
        err_rate = min(err01, err10)
        
        if(err_rate < train_err_rate):
            t = t_val
            train_err_rate = err_rate
            if (err_rate == err01):
                a=0
                b=1
            else:
                a=1
                b=0
    
    return (t, a, b, train_err_rate)

x = np.array([0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9 ])
y = np.array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1])
t, a, b, train_err = find_best_stump(x, y)
print(f'{(t,a,b)}, {train_err:.3}')

(0.75, 0, 1), 0.176


In [12]:
################################################################################
# Part (e)

# Function to generate data
def generate_data(n):
    x = np.random.rand(n)
    z = np.random.rand(n)
    y = np.zeros(n)
    y[x <= t1] = z[x <= t1] <= p0
    y[(x > t1) * (x < t2)] = z[(x > t1) * (x < t2)] <= p1
    y[x >= t2] = z[x >= t2] <= p2
    return (x,y)

# Simulation code
np.random.seed(42)
n = 100
num_trials = 5000
error_rates = np.zeros(num_trials)
thresholds = np.zeros(num_trials)
for trial in range(num_trials):
    t, a, b, _ = find_best_stump(*generate_data(n))
    thresholds[trial] = t
    error_rates[trial] = stump_err(t, a, b)

# Plotting code
plt.figure()
plt.hist(thresholds, bins=50)
plt.xlabel('$\hat\\theta$')
plt.ylabel('counts')
plt.savefig('histogram1.pdf', bbox_inches='tight')
plt.close()

plt.figure()
plt.hist(error_rates, bins=50)
plt.xlabel('error rate')
plt.ylabel('counts')
plt.savefig('histogram2.pdf', bbox_inches='tight')
plt.close()